# Creación de excels

## Importación de librerías

In [8]:
import random as rd
import openpyxl
import csv

## Funciones auxiliares

Funciones para generar RUT sacadas de [aquí](https://github.com/Mansilla1/python_rut).

In [7]:
def write_csv(nombre_archivo, lista_data, inicio=False):
    if inicio:
        tipo = 'w'
    else:
        tipo = 'a'
    with open(nombre_archivo, tipo) as output:
        writer = csv.writer(output, delimiter=";", quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
        for i in lista_data:
            writer.writerow([i])

# obtener el dígito verificador de un rut
def digito_verificador(rut):
    producto = [2,3,4,5,6,7] # producto de con el cual se debe multiplicar
    list_rut = list(map(int, str(rut))) # convertir en lista el rut
    list_rut.reverse() # revertir los valores
    contador = 0
    pivote = 0
    for i in list_rut:
        if pivote >= len(producto): # si el pivote pasa la cantidad del largo de producto, se debe reiniciar
            pivote = 0
        contador = contador+(i*producto[pivote])
        pivote += 1
    suma_dig = 11-(contador%11) # obtener el resto menos 11 de la suma
    # definir digito verificador
    if suma_dig == 11:
        verificador = 0
    elif suma_dig == 10:
        verificador = 'K'
    else:
        verificador = suma_dig

    return verificador

# método para generar ruts y exportar en archivo csv (opcional)
def genera_rut(**kwargs):
    # rut autoincremental a partir del numero indicado
    keys = []
    for key in kwargs.keys():
        keys.append(key)
    # cantidad de rut a generar
    if not 'cantidad' in keys: # si no se especifica la cantidad, se generarán 10
        cant_rut = 10
    else:
        cant_rut = int(kwargs['cantidad'])
    # inicio del rut (es autoincremental)
    if not 'inicio' in keys:
        inicio = 1
    else:
        inicio = int(kwargs['inicio'])
    # si se exporta o no
    if not 'csv' in keys:
        exportar = False
    else:
        exportar = kwargs['csv'] # true or false

    pivot = 0
    lista_rut = []
    while pivot < cant_rut:
        rut = str(inicio) + '-' + str(digito_verificador(inicio))
        lista_rut.append(rut)
        # autoincrementales
        inicio += 1
        pivot += 1

    if exportar:
        write_csv('output.csv', lista_rut, True)
    else:
        return lista_rut

## Orgánica

Creamos un libro con las columnas "RUT", "UR", "Cargo" y "Concat".

In [42]:
wb = openpyxl.Workbook()
hoja = wb.active
hoja.title = "Informe"
hoja.append(('RUT', 'UR', 'Cargo', 'Concat'))

Generamos *n* RUTS. Generamos las columnas requeridas para cada uno y lo guardamos en el libro. También guardamos las concats para utilizar luego.

In [43]:
n = 25
ruts = genera_rut(cantidad=n, inicio=10000000, csv=False)
concats = set()

for i in range(0,n):
  ur = str(rd.randint(1000,9999))
  cargo = str(rd.randint(10,9999))
  concat = ur + "-" + cargo
  concats.add(concat)
  hoja.append((ruts[i], ur, cargo, concat))

Guardamos el libro.

In [44]:
wb.save('organica.xlsx')

## Roles

Creamos un libro con las columnas "Rol", "Aplicación", "Perfil", "UR", "Cargo" y "Concat".

In [51]:
wb2 = openpyxl.Workbook()
hoja = wb2.active
hoja.title = "Informe"
hoja.append(('Rol', 'Aplicacion', 'Perfil', 'UR', 'Cargo', 'Concat'))

Para cada concatenación distinta, se creará un número aleatorio de accesos *m*. Por cada acceso se agrega una fila con las columnas requeridas.

In [52]:
for id,val in enumerate(concats):
  ur, cargo = val.split('-')
  m = rd.randint(0,10)
  [app, pf] = [rd.randint(0,10), rd.randint(0,3)]
  for j in range(0,m):
    hoja.append((f'Rol {id}', f'App {app+j}', f'Perfil {pf+j}', ur, cargo, val))

Guardamos el libro.

In [53]:
wb2.save('catalogo.xlsx')